In [1]:
##Setting up the python environment
import pandas as pd
import os 
import re
from collections import Counter
import numpy as np
import math
import matplotlib.pyplot as plt
import string
%matplotlib inline 
import textwrap

##Setting up BQ API
from google.cloud import bigquery

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


##Setting up Google sdk environment
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter/.config/gcloud/application_default_credentials.json'
## Connect to the right GCP project
os.environ['GCLOUD_PROJECT'] = 'som-nero-egyang-starr' 
%load_ext google.cloud.bigquery
client=bigquery.Client()

project_id = "som-rit-phi-starr-prod"
# dataset_id = "starr_omop_cdm5_deid_1pcent_latest"
dataset_id = "starr_omop_cdm5_deid_latest"
work_project_id = 'som-nero-egyang-starr' 
work_dataset_id = 'ilies_explore'

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## BMI missingness

In [4]:
query="""
CREATE OR REPLACE TABLE `{work_project_id}.{work_dataset_id}.person_ids_BMI_30` AS
SELECT * FROM
((SELECT a.* EXCEPT (PAD, Smoking_status), 1 AS label
FROM `{work_project_id}.{work_dataset_id}.Cases_FM_temp_smok` a
WHERE a.person_id NOT IN (
SELECT *
FROM `{work_project_id}.{work_dataset_id}.321052_ids`)
)
UNION ALL
(SELECT a.* EXCEPT (PAD, Smoking_status), 0 AS label
FROM `{work_project_id}.{work_dataset_id}.Controls_FM_temp_smok` a ))
WHERE BMI = 30
""".format_map({
                'project_id': project_id,
                'dataset_id': dataset_id,
                'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id})
df = client.query(query).to_dataframe()
df

""


In [6]:
total = 20031

In [7]:
BMI30 = 100

In [13]:
BMINULL = 13

In [14]:
total - BMI30 - BMINULL

19918

In [ ]:
fold_num = 1
query="""
(SELECT a.* EXCEPT (PAD, Smoking_status), 1 AS label
FROM `{work_project_id}.{work_dataset_id}.Cases_FM_temp_smok` a
JOIN `{work_project_id}.{work_dataset_id}.person_ids_cases_sequence_fold_{fold_num}` b
ON a.person_id = b.person_id
WHERE a.person_id NOT IN (
SELECT *
FROM `{work_project_id}.{work_dataset_id}.321052_ids`)
)
UNION ALL
(SELECT a.* EXCEPT (PAD, Smoking_status), 0 AS label
FROM `{work_project_id}.{work_dataset_id}.Controls_FM_temp_smok` a
JOIN `{work_project_id}.{work_dataset_id}.person_ids_controls_sequence_fold_{fold_num}` b
ON a.person_id = b.person_id)
""".format_map({
                'project_id': project_id,
                'dataset_id': dataset_id,
                'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
                'fold_num': fold_num})
df = client.query(query).to_dataframe()
df